In [45]:
from IPython import get_ipython


def type_of_script():
    """
    Detects and returns the type of python kernel
    :return: string 'jupyter' or 'ipython' or 'terminal'
    """
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return 'jupyter'
        if 'terminal' in ipy_str:
            return 'ipython'
    except:
        return 'terminal'


if type_of_script() == 'jupyter':
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm
    
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker
from matplotlib import colormaps
from matplotlib.colors import Normalize
import matplotlib.gridspec as gridspec
import cmasher as cmr

import numpy as np
import os, glob, atexit, signal
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=0
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle, ax_change_color
from importlib import reload
from copy import deepcopy
from multiprocessing import Pool, shared_memory, Value
from common_func import *

In [46]:
mode1 = 'nh'
database1 = f"/home/jeon/MissingSat/database/{mode1}"
iout1 = 1026
repo1, rurmode1, dp1 = mode2repo(mode1)
snap1 = uri.RamsesSnapshot(repo1, iout1, mode=rurmode1)
snap1s = uri.TimeSeries(snap1)
snap1s.read_iout_avail()
nout1 = snap1s.iout_avail['iout']; nout=nout1[nout1 <= iout1]
gals1 = uhmi.HaloMaker.load(snap1, galaxy=True, double_precision=dp1)
hals1 = uhmi.HaloMaker.load(snap1, galaxy=False, double_precision=dp1)

LG1 = pklload(f"{database1}/LocalGroup.pickle")
allsats1 = None; allsubs1 = None; states1 = None
keys1 = list(LG1.keys())
for key in keys1:
    sats = LG1[key]['sats']; subs = LG1[key]['subs']; real = LG1[key]['real']
    dink = real[real['state']=='dink']['hid']
    ind = isin(subs['id'], dink)
    subs['dink'][ind] = True; subs['dink'][~ind] = False
    state = np.zeros(len(subs), dtype='<U7')
    state[ind] = 'dink'; state[~ind] = 'pair'
    
    upair = real[real['state']=='upair']['hid']
    ind = isin(subs['id'], upair)
    state[ind] = 'upair'

    allsats1 = sats if allsats1 is None else np.hstack((allsats1, sats))
    allsubs1 = subs if allsubs1 is None else np.hstack((allsubs1, subs))
    states1 = state if states1 is None else np.hstack((states1, state))
argsort = np.argsort(allsubs1['id'])
allsubs1 = allsubs1[argsort]; states1 = states1[argsort]
dinks1 = allsubs1[states1 == 'dink']
pairs1 = allsubs1[states1 == 'pair']
upairs1 = allsubs1[states1 == 'upair']

print(len(allsubs1), np.unique(states1, return_counts=True))  

1336 (array(['dink', 'pair', 'upair'], dtype='<U7'), array([1145,  130,   61]))


In [47]:
mode2 = 'nh2'
database2 = f"/home/jeon/MissingSat/database/{mode2}"
iout2 = 797
repo2, rurmode2, dp2 = mode2repo(mode2)
snap2 = uri.RamsesSnapshot(repo2, iout2, mode=rurmode2)
snap2s = uri.TimeSeries(snap2)
snap2s.read_iout_avail()
nout2 = snap2s.iout_avail['iout']; nout=nout2[nout2 <= iout2]
gals2 = uhmi.HaloMaker.load(snap2, galaxy=True, double_precision=dp2)
hals2 = uhmi.HaloMaker.load(snap2, galaxy=False, double_precision=dp2)

LG2 = pklload(f"{database2}/LocalGroup.pickle")
allsats2 = None; allsubs2 = None; states2 = None
keys2 = list(LG2.keys())
for key in keys2:
    sats = LG2[key]['sats']; subs = LG2[key]['subs']; real = LG2[key]['real']
    dink = real[real['state']=='dink']['hid']
    ind = isin(subs['id'], dink)
    subs['dink'][ind] = True; subs['dink'][~ind] = False
    state = np.zeros(len(subs), dtype='<U7')
    state[ind] = 'dink'; state[~ind] = 'pair'
    
    upair = real[real['state']=='upair']['hid']
    ind = isin(subs['id'], upair)
    state[ind] = 'upair'

    allsats2 = sats if allsats2 is None else np.hstack((allsats2, sats))
    allsubs2 = subs if allsubs2 is None else np.hstack((allsubs2, subs))
    states2 = state if states2 is None else np.hstack((states2, state))
argsort = np.argsort(allsubs2['id'])
allsubs2 = allsubs2[argsort]; states2 = states2[argsort]
dinks2 = allsubs2[states2 == 'dink']
pairs2 = allsubs2[states2 == 'pair']
upairs2 = allsubs2[states2 == 'upair']

print(len(allsubs2), np.unique(states2, return_counts=True))

1242 (array(['dink', 'pair', 'upair'], dtype='<U7'), array([1120,   79,   43]))


In [48]:
# ptree_dm1 = pklload("/storage6/NewHorizon/ptree_dm/ptree_stable.pkl")
# lastouts1 = nout1[snap1s.iout_avail['age'] >= np.max(snap1s.iout_avail['age']-1)]
# ptree_dm1 = ptree_dm1[isin(ptree_dm1['timestep'], lastouts1)]
# halos1 = pklload(f"{database1}/halo_dict.pickle")
# prog_fromp1 = {}
# tmp = ptree_dm1[ptree_dm1['timestep']==1026]
# for sub in tqdm(allsubs1, desc='branch from ptree'):
#     last = tmp[tmp['hmid'] == sub['id']][0]['last']
#     branch = ptree_dm1[ptree_dm1['last'] == last]
    
#     mybranch = None
#     for ib in branch:
#         index = halos1['index'][ib['timestep']][ib['hmid']-1]
#         ihal = halos1['catalog'][ib['timestep']][index]
#         mybranch = ihal if(mybranch is None) else np.hstack((mybranch, ihal))
#     prog_fromp1[sub['id']] = mybranch
# pklsave(prog_fromp1, f"{database1}/06a_branch_from_ptree.pickle")

# count = 0
# shared_particles1 = {}
# for target in tqdm(allsubs1, desc='shared particle maximum'):
#     count += 1
#     progs = prog_fromp1[target['id']]
#     parts = uhmi.HaloMaker.read_member_general(snap1s, progs, galaxy=False)
#     ids, counts = np.unique(parts['id'], return_counts=True)
#     thresh = len(progs)
#     stables = np.array([], dtype=int)
#     while len(stables) < target['nparts']/10:
#         stables = ids[counts >= thresh]
#         thresh -= 1
#         if(thresh < 1):
#             break
#     shared_particles1[target['id']] = stables
#     if(count%100 == 0):
#         snap1s.clear()
# pklsave(shared_particles1, f"{database1}/06b_shared_particles.pickle")

In [49]:
ptree_dm2 = pklload("/storage7/NH2/ptree_dm/ptree_stable.pkl")
lastouts2 = nout2[snap2s.iout_avail['age'] >= np.max(snap2s.iout_avail['age']-1)]
ptree_dm2 = ptree_dm2[isin(ptree_dm2['timestep'], lastouts2)]
halos2 = pklload(f"{database2}/halo_dict.pickle")
prog_fromp2 = {}
tmp = ptree_dm2[ptree_dm2['timestep']==797]
for sub in tqdm(allsubs2, desc='branch from ptree'):
    last = tmp[tmp['hmid'] == sub['id']][0]['last']
    branch = ptree_dm2[ptree_dm2['last'] == last]
    
    mybranch = None
    for ib in branch:
        index = halos2['index'][ib['timestep']][ib['hmid']-1]
        ihal = halos2['catalog'][ib['timestep']][index]
        mybranch = ihal if(mybranch is None) else np.hstack((mybranch, ihal))
    prog_fromp2[sub['id']] = mybranch
pklsave(prog_fromp2, f"{database2}/06a_branch_from_ptree.pickle")

count = 0
shared_particles2 = {}
for target in tqdm(allsubs2, desc='shared particle maximum'):
    count += 1
    progs = prog_fromp2[target['id']]
    parts = uhmi.HaloMaker.read_member_general(snap2s, progs, galaxy=False)
    ids, counts = np.unique(parts['id'], return_counts=True)
    thresh = len(progs)
    stables = np.array([], dtype=int)
    while len(stables) < target['nparts']/10:
        stables = ids[counts >= thresh]
        thresh -= 1
        if(thresh < 1):
            break
    shared_particles2[target['id']] = stables
    if(count%100 == 0):
        snap2s.clear()
pklsave(shared_particles2, f"{database2}/06b_shared_particles.pickle")

branch from ptree:   0%|          | 0/1242 [00:00<?, ?it/s]

KeyError: 796

In [51]:
halos2

{'catalog': {14: rec.array([(140, 6, 14, 1, 6, 0, 0, -1, 0.05748538, 1.82729447e+08, 0.51306584, 0.4890063, 0.46625134, -8.24918635, 7.23328532, 45.53391674, 397786.26506453, -633008.35555813, 880460.7483499, 0.00089125, 0.0035703, 0.0003256, 0.0005046, 0.23988804, -0.50234684, -0.26245881, 0.09639205, 10.81488881, 7.06472073e-05, 33935468.64829074, 9083.18170054, 15.90639516, 140671.03410865, 7.06472073e-05, 0.),
             (140, 6, 14, 1, 6, 0, 0, -1, 0.05748538, 1.82729447e+08, 0.51306584, 0.4890063, 0.46625134, -8.24918635, 7.23328532, 45.53391674, 397786.26506453, -633008.35555813, 880460.7483499, 0.00089125, 0.0035703, 0.0003256, 0.0005046, 0.23988804, -0.50234684, -0.26245881, 0.09639205, 10.81488881, 7.06472073e-05, 33935468.64829074, 9083.18170054, 15.90639516, 140671.03410865, 7.06472073e-05, 0.),
             (140, 6, 14, 1, 6, 0, 0, -1, 0.05748538, 1.82729447e+08, 0.51306584, 0.4890063, 0.46625134, -8.24918635, 7.23328532, 45.53391674, 397786.26506453, -633008.35555813, 8

In [52]:
tree = pklload(f"{database2}/main_progenitors.pickle")
tree = tree[isin(tree['lastid'], allsubs2['id'])]

iout = 796
isnap = snap2s.get_snap(iout)
ihals = uhmi.HaloMaker.load(isnap, galaxy=False, double_precision=dp2)
indicies = np.zeros(len(ihals), dtype=int)
iids = tree[tree['timestep'] == iout]['id']
print(iids)
ihals = ihals[iids-1]
indicies[iids-1] = np.arange(len(iids))
halos2['catalog'][iout] = ihals
halos2['index'][iout] = indicies   
pklsave(halos2, f"{database2}/halo_dict.pickle")

[20997  1537  1587 ... 28948 28949 28951]


In [24]:
iouts = os.listdir("/storage7/NH2/halo")
iouts = [int(f[-5:]) for f in iouts if f.startswith("tree_bricks")]
iouts.sort()
iouts = np.array(iouts[::-1])
iouts = iouts[iouts < 797]
print(f"{np.max(iouts)} ~ {np.min(iouts)}")
nstep = len(iouts)
nsub = len(allsubs2)

796 ~ 14


In [25]:
nstep*nsub, len(tree)

(972486, 972486)

In [37]:
newtree = np.zeros((nstep+1)*nsub, dtype=tree.dtype)
newtree[:nsub] = tree[:nsub]
newtree[2*nsub:] = tree[nsub:]

In [38]:
lastids = {}
maxid = 0
for sub in tqdm( allsubs2 ):
    lastids[sub['id']] = uhmi.HaloMaker.read_member_part(snap2s.get_snap(sub['timestep']), sub['id'], galaxy=False, simple=True)
    maxid = max(maxid, lastids[sub['id']].max())

  0%|          | 0/1242 [00:00<?, ?it/s]

In [40]:
pout=796
cursor=nsub
psnap = snap2s.get_snap(pout)
phals, ppids = uhmi.HaloMaker.load(psnap, galaxy=False, load_parts=True)
cparts = phals['nparts']; cparts = np.cumsum(cparts); cparts = np.hstack((0, cparts))
table = np.repeat(phals['id'], phals['nparts']).astype(np.int16)
pids = np.zeros(max(maxid, ppids.max()), dtype=np.int32)
pids[ppids-1] = table

newtree[cursor:cursor+nsub]['lastid'] = allsubs2['id']
newtree[cursor:cursor+nsub]['timestep'] = pout

keys = list(lastids.keys())
for key in tqdm(keys):
    pid = lastids[key]

    # Find Prog
    progs = pids[pid-1]
    unique, counts = np.unique(progs, return_counts=True)
    if( (len(unique)==1)&(0 in unique) ):
        prog=0
        give_score=0
        take_score=0
    else:
        if(0 in unique):
            mask = unique!=0
            unique = unique[mask]; counts = counts[mask]
        
        # Give Score
        argmax = np.argmax(counts)
        prog = unique[argmax]
        give_score = counts[argmax] / len(pid)
        # print(prog, give_score)

        # Take Score
        part_of_prog = ppids[ cparts[prog-1]:cparts[prog] ]
        ind = isin(part_of_prog, pid, assume_unique=True)
        take_score = np.sum(ind)/len(part_of_prog)
    newtree[cursor]['id'] = prog
    newtree[cursor]['give_score'] = give_score
    newtree[cursor]['take_score'] = take_score
    cursor += 1
psnap.clear()

  0%|          | 0/1242 [00:00<?, ?it/s]

In [43]:
newtree[nsub:2*nsub]

array([(  644, 796, 20997, 0.96691176, 1.        ),
       ( 1529, 796,  1537, 0.98058252, 0.99019608),
       ( 1577, 796,  1587, 1.        , 0.99465241), ...,
       (28895, 796, 28948, 0.87739464, 0.94628099),
       (28896, 796, 28949, 0.90189873, 0.8662614 ),
       (28899, 796, 28951, 0.99232737, 0.8960739 )],
      dtype=[('lastid', '<i2'), ('timestep', '<i2'), ('id', '<i2'), ('give_score', '<f8'), ('take_score', '<f8')])

In [44]:
pklsave(newtree, f"{database2}/main_progenitors.pickle")

In [18]:
fnames = os.listdir(f"/storage7/NH2/halo/")
fnames = np.array([int(fname[-5:]) for fname in fnames if fname.startswith('tree_bricks')])
fnames[~isin(fnames, np.unique(tree['timestep']))]

array([796, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809,
       810, 811])

In [16]:
np.unique(tree['timestep'])

array([ 14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
       183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 19

In [5]:
halos2['index'].keys()

dict_keys([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 

In [6]:
halos2['catalog'][797].shape

(1242,)

In [23]:
halos1 = pklload(f"{database1}/halo_dict.pickle.old")

In [25]:
halos1['catalog'][1026]['id']

array([  589,   605,   610, ..., 28582, 28865, 28866], dtype=int32)

In [29]:
halos1['index'][1026][605-1]

1

In [31]:
prog_fromp1 = {}
tmp = ptree_dm1[ptree_dm1['timestep']==1026]
for sub in tqdm(allsubs1):
    last = tmp[tmp['hmid'] == sub['id']][0]['last']
    branch = ptree_dm1[ptree_dm1['last'] == last]
    
    mybranch = None
    for ib in branch:
        index = halos1['index'][ib['timestep']][ib['hmid']-1]
        ihal = halos1['catalog'][ib['timestep']][index]
        mybranch = ihal if(mybranch is None) else np.hstack((mybranch, ihal))
    prog_fromp1[sub['id']] = mybranch

  0%|          | 0/1336 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [42]:
allsubs1['id']

array([  693,  1309,  1310, ..., 28532, 28581, 28582], dtype=int32)

In [43]:
target = allsubs1[0]
progs = prog_fromp1[target['id']]
progs

array([(204,   690,  954, 1, 690,   0, 0,    -1, 0.81788361, 3.02808797e+08, 0.48285129, 0.46790566, 0.47471491, 134.88585641,  21.5432959 ,  88.8442938 , -4.61408049e+07, -1.44472804e+07,  2.68156854e+07, 0.00039413, 0.01610122, 0.00642717, 0.0070712 , 13.99594688, -0.14864931, 13.84729757,  9.48033857, 48.16790521, 1.05031910e-04, 1.56625240e+08, 1981.9174193 , 7.43011514, 68.60117311, 1.05031910e-04, 4.17667306e+07),
       (196,   678,  951, 1, 678,   0, 0,    -1, 0.81607634, 2.74094170e+08, 0.48280896, 0.46790373, 0.47470499, 138.82186199,  14.90510849,  77.31384962,  5.56183059e+06,  1.50787246e+07,  6.04999147e+06, 0.00041943, 0.01585448, 0.00407478, 0.00665236, 10.73924457, -0.13446497, 10.6047796 ,  3.30576842, 40.04573779, 1.07560921e-04, 1.67066923e+08, 2068.91068913, 7.59143081, 68.58713487, 1.07560921e-04, 2.08833653e+07),
       (200,   679,  952, 1, 679,   0, 0,    -1, 0.81693679, 2.97587956e+08, 0.48282806, 0.46790376, 0.47471099, 135.61549653,  20.66023014,  89.0379067

In [33]:
parts = uhmi.HaloMaker.read_member_general(snap1s, progs, galaxy=False)

In [40]:
np.sum(progs['nparts']), len(progs)

(23856, 128)

In [44]:
ids, counts = np.unique(parts['id'], return_counts=True)
thresh = len(progs)
stables = []
while len(stables) < target['nparts']/10:
    stables = ids[counts >= thresh]
    thresh -= 1
    if(thresh < 1):
        raise ValueError("No stable particles found")


In [46]:
stables.shape

(106,)